In [ ]:
%pip install ipyleaflet
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from ipyleaflet import Map, DrawControl, SearchControl, ZoomControl, Marker
import ipywidgets as widgets
from dabpy import *


In [3]:
# Replace with your WHOS API token and optional view
token = "whos-2bbd201b-9908-44b6-ad5c-e3b9a6e3bcda"  # replace with your actual token
view = "whos"
client = WHOSClient(token=token, view=view)

In [ ]:
# Map Visualization to select bounding box
bbox_holder = {}

# Create map centered on Firenze
m = Map(zoom=5, center=[43.7696, 11.2558], zoom_control=False)
m.add(ZoomControl(position='topright'))

# Add Search box below zoom
search = SearchControl(
    position="topright",  # will appear below zoom automatically
    url='https://nominatim.openstreetmap.org/search?format=json&q={s}',
    zoom=11,
    marker=Marker()
)
m.add(search)

# Draw rectangle only, without edit/delete buttons
draw_control = DrawControl(
    rectangle={
        "shapeOptions": {"color": "#ff0000", "weight": 2},
        "edit": False,
        "remove": False
    },
    polygon={}, polyline={}, circle={}, circlemarker={}, marker={}
)
m.add(draw_control)

output = widgets.Output()

def handle_draw(target, action, geo_json):
    if action == "created":
        draw_control.clear()  # only one rectangle

    coords = geo_json["geometry"]["coordinates"][0]
    lons = [c[0] for c in coords]
    lats = [c[1] for c in coords]

    south = min(lats)
    west  = min(lons)
    north = max(lats)
    east  = max(lons)

    bbox_holder["bbox"] = (south, west, north, east)

    with output:
        output.clear_output()
        print("Selected bounding box:")
        print(f"south={south}, west={west}, north={north}, east={east}")

draw_control.on_draw(handle_draw)

widgets.VBox([m, output])

In [7]:
# Ensure a bbox was selected
if "bbox" not in bbox_holder:
    raise ValueError("Please draw a bounding box on the map.")

constraints = Constraints(bbox=bbox_holder["bbox"])

In [8]:
## 01 GET FEATURES
features = client.get_features(constraints)
features_df = client.features_to_df(features)
display(features_df)

Retrieving https://whos.geodab.eu/gs-service/services/essi/token/***/view/whos/om-api/features?west=10.283203&south=42.261049&east=13.535156&north=44.245199


,ID,Name,Coordinates,Source,Identifier,Contact Name,Contact Email
0,002D9537CD8255D068986B7878C3DBA9D407AA6C,Montecatini Terme,"10.773986, 43.896797","Italy, Italian Institute for Environmental Pro...",cuahsi:hsl-tos:TOS01001601,,
1,0058B55ECAAC1EF95B945D36A626BDAE6518FDDE,MONTEGROSSO,"11.872, 44.072","Italy, Italian Institute for Environmental Pro...",cuahsi:hsl-emr:MONTEGROSSO,,
2,00944058DA00A5C310F8D3528C3A9E16F71D53BF,Vergemoli,"10.388588, 44.053029","Italy, Italian Institute for Environmental Pro...",cuahsi:hsl-tos:TOS10000288,,
3,009AD4B695D7FDD2AB1C598DB4D89857C8A1AC7C,Fano,"13.0538, 43.8263","Italy, Italian Institute for Environmental Pro...",cuahsi:hsl-mar:152,,
4,00BD3A024B5E7240B7657CA102A4785CCD13BD66,Stia,"11.718598, 43.802706","Italy, Italian Institute for Environmental Pro...",cuahsi:hsl-tos:TOS01004379,,
5,00E10205B57C60177B8D6375D68AE894CEE565EE,Jesi,"13.23, 43.5319","Italy, Italian Institute for Environmental Pro...",cuahsi:hsl-mar:110,,
6,00F711720986971352F2BED1DBD254DC3786A256,S. Colomba,"11.229133, 43.326058","Italy, Italian Institute for Environmental Pro...",cuahsi:hsl-tos:TOS10002758,,
7,01424E9BD5E6F61FC7B6492B6B72F2B716F9D4E8,Firenze Idrografico,"11.259264, 43.781494","Italy, Italian Institute for Environmental Pro...",cuahsi:hsl-tos:TOS10001080,,
8,01B7E5018C60A3B9A4443FC9238B340B82A5E2F6,Boccheggiano,"11.035925, 43.091793","Italy, Italian Institute for Environmental Pro...",cuahsi:hsl-tos:TOS10002710,,
9,01B875F80C1B6F4E3D6C4D90E5EA4740D6821505,Pontecosi,"10.38436, 44.13402","Italy, Italian Institute for Environmental Pro...",cuahsi:hsl-tos:TOS10000210,,


In [9]:
## 02 GET OBSERVATIONS
feature_used = features[4]
feature_id = feature_used.id
observations = client.get_observations(feature_id)

observations_df = client.observations_to_df(observations)
if observations_df is not None:
    display(observations_df)

## 03 GET DATA POINTS
obs_with_data = client.get_observation_with_data(
    observations[0].id,
    begin="2025-01-01T00:00:00Z",
    end="2025-02-01T00:00:00Z"
)

if obs_with_data:
    obs_points_df = client.points_to_df(obs_with_data)
    display(obs_points_df)
    client.plot_observation(obs_with_data, feature=feature_used)
else:
    print("No observation data available.")


Retrieving https://whos.geodab.eu/gs-service/services/essi/token/***/view/whos/om-api/observations?feature=00BD3A024B5E7240B7657CA102A4785CCD13BD66


,ID,Source,Observed Property Definition,Original Observed Property,Observed Property,Phenomenon Time Begin,Phenomenon Time End,Feature of Interest Href,Observation Type,Unit of Measurement,Interpolation Type
0,A3DD5E40EB4AB1A6406D9266D37A46B7592FE404,"Italy, Italian Institute for Environmental Pro...",Discharge Data type: Average Value type: Field...,Discharge,"Flux, discharge",1939-12-31T23:00:00Z,2013-12-30T23:00:00Z,00BD3A024B5E7240B7657CA102A4785CCD13BD66,TimeSeriesObservation,cubic meters per second,Average
1,E8375411442EE4402EE17ACB62F52591F037B595,"Italy, Italian Institute for Environmental Pro...",Water Level Data type: Unknown Value type: Fie...,Water Level,Level,1997-12-31T23:00:00Z,2015-03-30T23:00:00Z,00BD3A024B5E7240B7657CA102A4785CCD13BD66,TimeSeriesObservation,meter,None
2,EF9D8DA0082509098502C8A0B0F5D82A7F3943F9,"Italy, Italian Institute for Environmental Pro...",Precipitation Data type: Cumulative Value type...,Precipitation,Precipitation,1991-11-02T08:00:00Z,2015-03-28T08:00:00Z,00BD3A024B5E7240B7657CA102A4785CCD13BD66,TimeSeriesObservation,millimeter,Total


Retrieving https://whos.geodab.eu/gs-service/services/essi/token/***/view/whos/om-api/observations?includeData=true&observationIdentifier=A3DD5E40EB4AB1A6406D9266D37A46B7592FE404&beginPosition=2025-01-01T00%3A00%3A00Z&endPosition=2025-02-01T00%3A00%3A00Z
No observation data available for the requested time range.
No observation data available.
